In [7]:
import os
from dotenv import load_dotenv
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationStringBufferMemory
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from midiutil import MIDIFile
import mido

In [3]:
load_dotenv()

openai.api_key = os.environ["OPENAI_API_KEY"]

### Import instruction

In [150]:
import omegaconf

# Load the config file
config = omegaconf.OmegaConf.load("../config/main.yaml")

In [151]:
instructions = config['prompt_text_to_melody']

In [152]:
instructions[0:100]

'I need assistance in producing AI-generated text that I convert to music using MIDI files. Initially'

### Setup the Model

In [153]:
temperature=0.9
llm_model = "gpt-4"

In [154]:
llm = ChatOpenAI(temperature=temperature, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [155]:
conversation.predict(input=instructions)

'YES'

In [384]:
composition = conversation.predict(input="""
    Yes, generate a longer melody very sad and dramatic.
    """)
composition

'Certainly, I can generate a longer "sad" and "dramatic" melody for you. Sad and dramatic melodies often involve minor keys, slower rhythms, larger leaps in intervals, and varied dynamics. Here is a more extended melody with these qualities:\n\n```python\nmelody_pitch_duration_data = [\n  (57, 0.5),   # A3 half note\n  (0, 0.125),   # Silence\n  (56, 0.5),  # G#3 half note\n  (57, 0.5),   # A3 half note\n  (0, 0.125),  # Silence \n  (55, 0.5),  # G3 half note\n  (57, 0.5),   # A3 half note\n  (0, 0.125),  # Silence \n  (54, 0.75),  # F#3 dotted half note\n  (0, 0.125),  # Silence\n  (52, 0.5),   # E3 half note\n  (0, 0.125),  # Silence\n  (50, 0.5),  # D3 half note\n  (52, 0.5),   # E3 half note\n  (0, 0.125),  # Silence\n  (54, 0.5),  # F#3 half note\n  (55, 0.5),   # G3 half note\n  (0, 0.125),  # Silence\n  (57, 0.5),  # A3 half note\n  (55, 0.5),   # G3 half note\n  (0, 0.125),  # Silence\n  (54, 0.5),  # F#3 half note\n  (52, 0.75),  # E3 dotted half note\n  (0, 0.125),  # Silence

### Parse the LLM output string into a Python dictionary

Let's create an LLM output JSON and use LangChain to parse that output.
Let's extract the melody data and format that output into a JSON format. 

In [385]:
description_schema = ResponseSchema(name="description",
                             description="Extract any sentences describing the melody \
                            an output them as a text.")
melody_pitch_duration_data_schema = ResponseSchema(name="melody_pitch_duration_data",
                                    description="Extract the composed melody, \
                                    and output them as a comma separated Python list like \
                                    [[60, 0.125], [62, 0.25], ...]")

response_schemas = [description_schema, 
                    melody_pitch_duration_data_schema]

In [386]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [387]:
format_instructions = output_parser.get_format_instructions()

In [388]:
review_template = """\
For the following text, extract the following information:

description: Extract any sentences describing the melody \
an output them as a text.

melody_pitch_duration_data: Extract the composed melody, \
and output them as a comma separated Python list. like \
[[60, 0.125], [62, 0.25], ...]

Format the output as JSON with the following keys:
description
melody_pitch_duration_data

text: {text}

{format_instructions}
"""

In [389]:
prompt = ChatPromptTemplate.from_template(template=review_template)

messages = prompt.format_messages(text=composition, 
                                format_instructions=format_instructions)

In [390]:
chat = ChatOpenAI(temperature=0.0, model=llm_model)

In [391]:
response = chat(messages)

In [392]:
print(response.content)

```json
{
	"description": "Sad and dramatic melodies often involve minor keys, slower rhythms, larger leaps in intervals, and varied dynamics. This melody starts on A3 (57) and ends on C3 (48), using multiple pitches, some in large intervals and others in small steps. The melody employs a minor scale with semitone shifts for added dissonance, which contributes to the sad and dramatic mood. The rhythm consists of half notes and dotted half notes interspersed with silences. This should give a sad, dramatic and complex sound.",
	"melody_pitch_duration_data": [[57, 0.5], [0, 0.125], [56, 0.5], [57, 0.5], [0, 0.125], [55, 0.5], [57, 0.5], [0, 0.125], [54, 0.75], [0, 0.125], [52, 0.5], [0, 0.125], [50, 0.5], [52, 0.5], [0, 0.125], [54, 0.5], [55, 0.5], [0, 0.125], [57, 0.5], [55, 0.5], [0, 0.125], [54, 0.5], [52, 0.75], [0, 0.125], [50, 0.5], [0, 0.125], [48, 0.5], [50, 0.5], [48, 0.75]]
}
```


In [393]:
output_dict = output_parser.parse(response.content)

In [394]:
output_dict

{'description': 'Sad and dramatic melodies often involve minor keys, slower rhythms, larger leaps in intervals, and varied dynamics. This melody starts on A3 (57) and ends on C3 (48), using multiple pitches, some in large intervals and others in small steps. The melody employs a minor scale with semitone shifts for added dissonance, which contributes to the sad and dramatic mood. The rhythm consists of half notes and dotted half notes interspersed with silences. This should give a sad, dramatic and complex sound.',
 'melody_pitch_duration_data': [[57, 0.5],
  [0, 0.125],
  [56, 0.5],
  [57, 0.5],
  [0, 0.125],
  [55, 0.5],
  [57, 0.5],
  [0, 0.125],
  [54, 0.75],
  [0, 0.125],
  [52, 0.5],
  [0, 0.125],
  [50, 0.5],
  [52, 0.5],
  [0, 0.125],
  [54, 0.5],
  [55, 0.5],
  [0, 0.125],
  [57, 0.5],
  [55, 0.5],
  [0, 0.125],
  [54, 0.5],
  [52, 0.75],
  [0, 0.125],
  [50, 0.5],
  [0, 0.125],
  [48, 0.5],
  [50, 0.5],
  [48, 0.75]]}

In [395]:
melody_data = output_dict.get('melody_pitch_duration_data')
melody_data

[[57, 0.5],
 [0, 0.125],
 [56, 0.5],
 [57, 0.5],
 [0, 0.125],
 [55, 0.5],
 [57, 0.5],
 [0, 0.125],
 [54, 0.75],
 [0, 0.125],
 [52, 0.5],
 [0, 0.125],
 [50, 0.5],
 [52, 0.5],
 [0, 0.125],
 [54, 0.5],
 [55, 0.5],
 [0, 0.125],
 [57, 0.5],
 [55, 0.5],
 [0, 0.125],
 [54, 0.5],
 [52, 0.75],
 [0, 0.125],
 [50, 0.5],
 [0, 0.125],
 [48, 0.5],
 [50, 0.5],
 [48, 0.75]]

In [396]:
note_to_midi = config['note_to_midi']

In [397]:
note_to_midi

{'A#0': 22, 'A#1': 34, 'A#2': 46, 'A#3': 58, 'A#4': 70, 'A#5': 82, 'A#6': 94, 'A#7': 106, 'A#8': 118, 'A0': 21, 'A1': 33, 'A2': 45, 'A3': 57, 'A4': 69, 'A5': 81, 'A6': 93, 'A7': 105, 'A8': 117, 'Ab0': 20, 'Ab1': 32, 'Ab2': 44, 'Ab3': 56, 'Ab4': 68, 'Ab5': 80, 'Ab6': 92, 'Ab7': 104, 'Ab8': 116, 'B0': 23, 'B1': 35, 'B2': 47, 'B3': 59, 'B4': 71, 'B5': 83, 'B6': 95, 'B7': 107, 'B8': 119, 'Bb0': 22, 'Bb1': 34, 'Bb2': 46, 'Bb3': 58, 'Bb4': 70, 'Bb5': 82, 'Bb6': 94, 'Bb7': 106, 'Bb8': 118, 'C#0': 13, 'C#1': 25, 'C#2': 37, 'C#3': 49, 'C#4': 61, 'C#5': 73, 'C#6': 85, 'C#7': 97, 'C#8': 109, 'C#9': 121, 'C0': 12, 'C1': 24, 'C2': 36, 'C3': 48, 'C4': 60, 'C5': 72, 'C6': 84, 'C7': 96, 'C8': 108, 'C9': 120, 'D#0': 15, 'D#1': 27, 'D#2': 39, 'D#3': 51, 'D#4': 63, 'D#5': 75, 'D#6': 87, 'D#7': 99, 'D#8': 111, 'D#9': 123, 'D0': 14, 'D1': 26, 'D2': 38, 'D3': 50, 'D4': 62, 'D5': 74, 'D6': 86, 'D7': 98, 'D8': 110, 'D9': 122, 'Db0': 13, 'Db1': 25, 'Db2': 37, 'Db3': 49, 'Db4': 61, 'Db5': 73, 'Db6': 85, 'Db7': 

In [398]:
type(melody_data)

list

In [399]:
from midiutil import MIDIFile

In [400]:
def note_to_midi_mapping():
    """
    Create a dictionary to map note names to MIDI note numbers.
    
    Returns:
        dict: A dictionary mapping note names to MIDI note numbers.
    """
    # Define the mapping from note names to MIDI note numbers
    note_to_midi = config['note_to_midi']
    return note_to_midi

In [401]:
def create_midi_file(notes, filename="melody.mid"):
    """
    Create a MIDI file from a list of notes.
    
    Parameters:
        notes (list): List of notes to include in the MIDI file.
        filename (str): Name of the output MIDI file.
    """
    # Initialize a MIDI file with one track
    midi = MIDIFile(1)
    
    # Set the instrument to Acoustic Grand Piano (program number 0)
    # The order is track, time, channel, program
    midi.addProgramChange(0, 0, 0, 0)
    
    # Add notes to the MIDI file
    for start_time, (pitch, duration) in enumerate(notes):
        midi.addNote(0, 0, pitch, start_time, duration, 100)
        
    # Save the MIDI file
    with open(filename, "wb") as output_file:
        midi.writeFile(output_file)



In [402]:
melody_data

[[57, 0.5],
 [0, 0.125],
 [56, 0.5],
 [57, 0.5],
 [0, 0.125],
 [55, 0.5],
 [57, 0.5],
 [0, 0.125],
 [54, 0.75],
 [0, 0.125],
 [52, 0.5],
 [0, 0.125],
 [50, 0.5],
 [52, 0.5],
 [0, 0.125],
 [54, 0.5],
 [55, 0.5],
 [0, 0.125],
 [57, 0.5],
 [55, 0.5],
 [0, 0.125],
 [54, 0.5],
 [52, 0.75],
 [0, 0.125],
 [50, 0.5],
 [0, 0.125],
 [48, 0.5],
 [50, 0.5],
 [48, 0.75]]

In [408]:
import ast

# Assuming melody_data is a string like '[[76, 0.125], [72, 0.125], ...]'
melody_data_list = ast.literal_eval(melody_data)

# Now melody_data_list should be a list of lists
print(melody_data_list)


ValueError: malformed node or string: [[57, 0.5], [0, 0.125], [56, 0.5], [57, 0.5], [0, 0.125], [55, 0.5], [57, 0.5], [0, 0.125], [54, 0.75], [0, 0.125], [52, 0.5], [0, 0.125], [50, 0.5], [52, 0.5], [0, 0.125], [54, 0.5], [55, 0.5], [0, 0.125], [57, 0.5], [55, 0.5], [0, 0.125], [54, 0.5], [52, 0.75], [0, 0.125], [50, 0.5], [0, 0.125], [48, 0.5], [50, 0.5], [48, 0.75]]

In [422]:
melody_data_double = melody_data + melody_data +  melody_data + melody_data
melody_data_double

[[57, 0.5],
 [0, 0.125],
 [56, 0.5],
 [57, 0.5],
 [0, 0.125],
 [55, 0.5],
 [57, 0.5],
 [0, 0.125],
 [54, 0.75],
 [0, 0.125],
 [52, 0.5],
 [0, 0.125],
 [50, 0.5],
 [52, 0.5],
 [0, 0.125],
 [54, 0.5],
 [55, 0.5],
 [0, 0.125],
 [57, 0.5],
 [55, 0.5],
 [0, 0.125],
 [54, 0.5],
 [52, 0.75],
 [0, 0.125],
 [50, 0.5],
 [0, 0.125],
 [48, 0.5],
 [50, 0.5],
 [48, 0.75],
 [57, 0.5],
 [0, 0.125],
 [56, 0.5],
 [57, 0.5],
 [0, 0.125],
 [55, 0.5],
 [57, 0.5],
 [0, 0.125],
 [54, 0.75],
 [0, 0.125],
 [52, 0.5],
 [0, 0.125],
 [50, 0.5],
 [52, 0.5],
 [0, 0.125],
 [54, 0.5],
 [55, 0.5],
 [0, 0.125],
 [57, 0.5],
 [55, 0.5],
 [0, 0.125],
 [54, 0.5],
 [52, 0.75],
 [0, 0.125],
 [50, 0.5],
 [0, 0.125],
 [48, 0.5],
 [50, 0.5],
 [48, 0.75],
 [57, 0.5],
 [0, 0.125],
 [56, 0.5],
 [57, 0.5],
 [0, 0.125],
 [55, 0.5],
 [57, 0.5],
 [0, 0.125],
 [54, 0.75],
 [0, 0.125],
 [52, 0.5],
 [0, 0.125],
 [50, 0.5],
 [52, 0.5],
 [0, 0.125],
 [54, 0.5],
 [55, 0.5],
 [0, 0.125],
 [57, 0.5],
 [55, 0.5],
 [0, 0.125],
 [54, 0.5],
 [52,

In [423]:
# Create and save the MIDI file
create_midi_file(melody_data)

### Reproduce a MIDI file in GarageBand on macOS using Python

Here is a complete guide for sending MIDI to GarageBand from Python using mido:

#### Send MIDI to GarageBand from Python

This guide covers how to reproduce a MIDI file in GarageBand on macOS using Python and the mido module.

##### Steps

1. Set up MIDI in Audio MIDI Setup

- Open Applications
- Open Audio MIDI Setup
- Enable the IAC Driver
- Connect a MIDI keyboard and test  

2. Create a Software Instrument track in GarageBand

- Launch GarageBand
- Create a new Software Instrument track 
- Select a sound like piano or synth 

3. Install mido

```bash
pip install mido
```

4. Import mido and get GarageBand port name

```python
import mido

ports = mido.get_output_names()
# Look for the GarageBand port, e.g. 'IAC Driver Bus 1'
gb_port = ports[0] 
```

5. Open MIDI file and GarageBand port

```python
mid = mido.MidiFile('melody.mid')

port = mido.open_output(gb_port)
```

6. Send MIDI messages to GarageBand

```python 
for message in mid.play():
    port.send(message)
```

7. Listen to the MIDI file reproduced in GarageBand!

#### Explanation

- Audio MIDI Setup enables virtual MIDI ports like the IAC Driver
- GarageBand needs an instrument track to receive the MIDI notes
- mido parses the MIDI file and sends messages to the port  
- The port name must match GarageBand's exactly

And that's it! The MIDI file will be played in GarageBand using the sounds from the instrument track. Let me know if you have any other questions!

In [4]:
# Import mido and get GarageBand port name
ports = mido.get_output_names()
# Look for the GarageBand port, e.g. 'IAC Driver Bus 1'
gb_port = ports[0]

In [5]:
# Open MIDI file and GarageBand port
mid = mido.MidiFile('melody.mid')
port = mido.open_output(gb_port)

FileNotFoundError: [Errno 2] No such file or directory: 'melody.mid'

In [6]:
# Send MIDI messages to GarageBand
for message in mid.play():
    port.send(message)

NameError: name 'mid' is not defined